# KIM 量表姿勢辨識

## 1. 所需套件

In [1]:
# 主要套件
import torch

# 資料處理
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# 建立機器模型
from torch import nn
from torch import optim

## 2. 所需參數

In [2]:
# 資料處理
csv_file = "dataset/train_data-x_y.csv"

# 資料處理
batch_size = 64

# 模型訓練
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu" # 取得 CPU 或是 GPU 的設備進行訓練
optim_fn = optim.SGD # 優化函數
loss_fn = nn.CrossEntropyLoss() # 損失函數
learning_rate = 1e-3 # 學習率
epochs = 5

print(f"Using {device} device")

Using cuda device


## 3. 從 csv 檔案中提取資料

In [3]:
kim_lhc_df = pd.read_csv(csv_file)

### 練習

#### 1. 顯示第一項資料

In [4]:
kim_lhc_df.iloc[0]

NOSE_x               494
NOSE_y               256
LEFT_EYE_x           517
LEFT_EYE_y           240
RIGHT_EYE_x          468
RIGHT_EYE_y          241
LEFT_EAR_x           553
LEFT_EAR_y           271
RIGHT_EAR_x          439
RIGHT_EAR_y          273
LEFT_SHOULDER_x      626
LEFT_SHOULDER_y      430
RIGHT_SHOULDER_x     386
RIGHT_SHOULDER_y     436
LEFT_ELBOW_x         644
LEFT_ELBOW_y         616
RIGHT_ELBOW_x        345
RIGHT_ELBOW_y        611
LEFT_WRIST_x         629
LEFT_WRIST_y         648
RIGHT_WRIST_x        395
RIGHT_WRIST_y        650
LEFT_HIP_x           563
LEFT_HIP_y           793
RIGHT_HIP_x          427
RIGHT_HIP_y          792
LEFT_KNEE_x          556
LEFT_KNEE_y         1064
RIGHT_KNEE_x         417
RIGHT_KNEE_y        1067
LEFT_ANKLE_x         546
LEFT_ANKLE_y        1289
RIGHT_ANKLE_x        425
RIGHT_ANKLE_y       1293
class_no               0
Name: 0, dtype: int64

#### 2. 將資料轉換成 tensor 格式

In [5]:
torch.tensor(list(kim_lhc_df.iloc[0]))

tensor([ 494,  256,  517,  240,  468,  241,  553,  271,  439,  273,  626,  430,
         386,  436,  644,  616,  345,  611,  629,  648,  395,  650,  563,  793,
         427,  792,  556, 1064,  417, 1067,  546, 1289,  425, 1293,    0])

#### 3. 顯示前五項資料

In [6]:
kim_lhc_df.head(5)

,NOSE_x,NOSE_y,LEFT_EYE_x,LEFT_EYE_y,RIGHT_EYE_x,RIGHT_EYE_y,LEFT_EAR_x,LEFT_EAR_y,RIGHT_EAR_x,RIGHT_EAR_y,...,RIGHT_HIP_y,LEFT_KNEE_x,LEFT_KNEE_y,RIGHT_KNEE_x,RIGHT_KNEE_y,LEFT_ANKLE_x,LEFT_ANKLE_y,RIGHT_ANKLE_x,RIGHT_ANKLE_y,class_no
0,494,256,517,240,468,241,553,271,439,273,...,792,556,1064,417,1067,546,1289,425,1293,0
1,491,262,513,245,465,247,548,274,437,279,...,798,553,1064,415,1069,543,1291,420,1294,0
2,488,261,510,245,462,247,546,274,433,279,...,797,551,1065,411,1067,539,1291,417,1295,0
3,485,254,508,237,460,240,549,267,436,275,...,787,553,1057,415,1063,541,1284,423,1288,0
4,484,262,507,246,458,247,543,277,430,280,...,798,549,1068,409,1069,538,1293,417,1295,0


#### 4. 顯示檔案中的索引資料以及數量

In [7]:
kim_lhc_index = kim_lhc_df.columns.to_list()
for i in range(len(kim_lhc_index)):
    print(f"{i}: {kim_lhc_index[i]}")

0: NOSE_x
1: NOSE_y
2: LEFT_EYE_x
3: LEFT_EYE_y
4: RIGHT_EYE_x
5: RIGHT_EYE_y
6: LEFT_EAR_x
7: LEFT_EAR_y
8: RIGHT_EAR_x
9: RIGHT_EAR_y
10: LEFT_SHOULDER_x
11: LEFT_SHOULDER_y
12: RIGHT_SHOULDER_x
13: RIGHT_SHOULDER_y
14: LEFT_ELBOW_x
15: LEFT_ELBOW_y
16: RIGHT_ELBOW_x
17: RIGHT_ELBOW_y
18: LEFT_WRIST_x
19: LEFT_WRIST_y
20: RIGHT_WRIST_x
21: RIGHT_WRIST_y
22: LEFT_HIP_x
23: LEFT_HIP_y
24: RIGHT_HIP_x
25: RIGHT_HIP_y
26: LEFT_KNEE_x
27: LEFT_KNEE_y
28: RIGHT_KNEE_x
29: RIGHT_KNEE_y
30: LEFT_ANKLE_x
31: LEFT_ANKLE_y
32: RIGHT_ANKLE_x
33: RIGHT_ANKLE_y
34: class_no


#### 5. 練習 pandas 的索引搜尋

In [8]:
label = tuple(kim_lhc_df.columns)
label[-1]

'class_no'

#### 清除變數

In [9]:
del(kim_lhc_index)
del(label)

## 4. 定義類別

### 3.1 定義資料集類別

In [10]:
# KIM 資料集
class KIMDataset(Dataset):
    def __init__(self, df: pd.DataFrame):
        """
        df: 儲存 pandas 的 DataFrame 格式的資料
        """
        # 儲存資料
        self.dataframe = df
        
        # 取得標籤
        self.label = tuple(df.columns)
        print(f"標籤數量: {len(self.label)}")
        # print("標籤內容:")
        # for i in range(len(self.label)):
        #     if i % 6 == 0 and i > 0:
        #         print("")
        #     print(self.label[i], end='\t')
        # print("\n")
        print(f"物件數量: {len(self)}")
        print("成功建立物件\n")
        pass
    
    def __getitem__(self, idx):
        """取得索引物件"""
        df_tensor = torch.tensor(self.dataframe.iloc[idx, :-1])
        return df_tensor, self.dataframe.iloc[idx, -1]
    
    def __len__(self):
        """取得資料集數量"""
        return len(self.dataframe)

#### 練習

##### 1. 建立資料集

In [11]:
kim_ds = KIMDataset(kim_lhc_df)

標籤數量: 35
物件數量: 3536
成功建立物件



##### 2. 顯示資料集中資料

In [12]:
kim_ds[0]

(tensor([ 494,  256,  517,  240,  468,  241,  553,  271,  439,  273,  626,  430,
          386,  436,  644,  616,  345,  611,  629,  648,  395,  650,  563,  793,
          427,  792,  556, 1064,  417, 1067,  546, 1289,  425, 1293]),
 0)

##### 3. 顯示資料集長度

In [13]:
len(kim_ds)

3536

##### 清除變數

In [14]:
del(kim_ds)

### 3.2 定義姿勢模型類別

In [15]:
# 姿勢模型
class BodyClassifyModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.relu_linear_stack = nn.Sequential(
            nn.Linear(34, 512, dtype=torch.int64),
            nn.ReLU(),
            nn.Linear(512, 4)
        )
        pass
    
    def forward(self, x):
        logits = self.relu_linear_stack(x)
        return logits

## 5. 將資料分為訓練資料集及測試資料集

In [16]:
train_df = kim_lhc_df.iloc[::2]
test_df = kim_lhc_df.iloc[1::2]

## 6. 分別建立資料集

In [17]:
train_ds = KIMDataset(train_df)
test_ds = KIMDataset(test_df)

標籤數量: 35
物件數量: 1768
成功建立物件

標籤數量: 35
物件數量: 1768
成功建立物件



## 7. 分別建立資料加載器

In [18]:
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

### 練習

#### 1. 顯示 DataLoader 裡面的資料

In [19]:
for X, y in train_dataloader:
    print(X)
    print(y)
    break

tensor([[ 494,  256,  517,  ..., 1289,  425, 1293],
        [ 488,  261,  510,  ..., 1291,  417, 1295],
        [ 484,  262,  507,  ..., 1293,  417, 1295],
        ...,
        [ 501,  396,  503,  ..., 1247,  385, 1323],
        [ 452,  378,  452,  ..., 1327,  284, 1326],
        [ 504,  381,  509,  ..., 1252,  394, 1272]])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


## 8. 建立模型物件

In [20]:
model = BodyClassifyModel().to(device)
model

RuntimeError: Only Tensors of floating point and complex dtype can require gradients

### 練習

#### 1. 取得練習範例中的資訊

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_DL = DataLoader(training_data, batch_size=64)
test_DL = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
test_model = NeuralNetwork().to(device)
print(test_model)

In [ ]:
for batch, (X, y) in enumerate(train_DL):
    X, y = X.to(device), y.to(device)
    print(X.shape)
    break

In [ ]:
del(training_data)
del(test_data)
del(train_DL)
del(test_DL)
del(test_model)

#### 1. 驗證模型是否正確

In [ ]:
for batch, (X, y) in enumerate(train_dataloader):
    X, y = X.to(device), y.to(device)
    print(X.dtype)
    # 模型輸入的 datatype 要跟我們的參數相同，否則無法使用
    
    print(model(X))
    # 計算預測失誤
    # pred = model(X)
    # print(pred)
    break

## 9. 建立優化模型函數

### 9.1 建立訓練函數

In [ ]:
def train(dataloader:DataLoader, model:nn.Module, loss_fn:nn.modules.loss._Loss, optimizer:optim.Optimizer):
    size = len(dataloader)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 計算預測失誤
        pred = model(X)
        loss = loss_fn(pred, y)
        print(type(loss))
        
        # Backpropagation(反向傳播算法)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

### 9.2 建立驗證函數

In [ ]:
def test(dataloader:DataLoader, model:nn.Module, loss_fn:nn.modules.loss._Loss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):0.1f}%, Avg loss: {test_loss:>8f} \n")

## 10. 進行訓練

In [ ]:
optimizer = optim_fn(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    
print("Done!")